In [86]:
import os
from os.path import join
import matplotlib.pyplot as plt
from random import randint
import pickle
import re #pre-process tag data
import pandas as pd
import numpy as np

firstposition = os.getcwd()
print(os.getcwd())

C:\Users\dbw21\Desktop\jupyter-notebook


In [3]:
photo_path = os.path.join(os.getcwd(),'Photos') 
print(len(os.listdir(photo_path))) #4851 #127 = 4978

4978


## read CSV file

In [5]:
csv_path = os.path.join(os.getcwd(),'Photo (1).csv')
meta_data = pd.read_csv(csv_path,encoding='cp949', names = ['photo_ID','user_ID','latitude','longitude','datetime','-1'])

In [6]:
meta_data #991766 rows × 6 columns

,photo_ID,user_ID,latitude,longitude,datetime,-1
0,8230341121,24691457@N00,47.610007,-122.340823,11/30/2012 12:22:19 AM,-1
1,8230337045,41132143@N00,47.520077,-122.297433,11/30/2012 12:20:08 AM,-1
2,8231412802,24691457@N00,47.611099,-122.337157,11/30/2012 12:27:15 AM,-1
3,8230344741,24691457@N00,47.611099,-122.337157,11/30/2012 12:24:21 AM,-1
4,8231415454,24691457@N00,47.611099,-122.337157,11/30/2012 12:28:40 AM,-1
...,...,...,...,...,...,...
991761,5898739929,47919464@N07,47.654976,-122.301175,7/3/2011 11:48:22 PM,-1
991762,5899308576,29296993@N02,47.613987,-122.341397,7/3/2011 11:49:47 PM,-1
991763,5899306798,29296993@N02,47.613987,-122.341397,7/3/2011 11:49:04 PM,-1
991764,5899303004,29296993@N02,47.613987,-122.341397,7/3/2011 11:47:27 PM,-1


## Preprocess CSV file 

* photo_ID 중복 행 제거
* photos 폴더 안에 있는 photoID 행만 남기기

In [7]:
#photo_ID 중복 행 제거
rmdup_metadata = meta_data.drop_duplicates(subset = 'photo_ID', keep='first', ignore_index = True)

In [8]:
rmdup_metadata #990696 rows × 6 columns

,photo_ID,user_ID,latitude,longitude,datetime,-1
0,8230341121,24691457@N00,47.610007,-122.340823,11/30/2012 12:22:19 AM,-1
1,8230337045,41132143@N00,47.520077,-122.297433,11/30/2012 12:20:08 AM,-1
2,8231412802,24691457@N00,47.611099,-122.337157,11/30/2012 12:27:15 AM,-1
3,8230344741,24691457@N00,47.611099,-122.337157,11/30/2012 12:24:21 AM,-1
4,8231415454,24691457@N00,47.611099,-122.337157,11/30/2012 12:28:40 AM,-1
...,...,...,...,...,...,...
990691,5898739929,47919464@N07,47.654976,-122.301175,7/3/2011 11:48:22 PM,-1
990692,5899308576,29296993@N02,47.613987,-122.341397,7/3/2011 11:49:47 PM,-1
990693,5899306798,29296993@N02,47.613987,-122.341397,7/3/2011 11:49:04 PM,-1
990694,5899303004,29296993@N02,47.613987,-122.341397,7/3/2011 11:47:27 PM,-1


### dictionary(photo_ID <-> index in dataframe)

In [9]:
photo_IDs = rmdup_metadata.photo_ID.values
#print(type(photo_IDs))

#dictionary: index(key) --> photo_ID(value) 
index_to_photoID = {}
for i in range(0,len(photo_IDs)):
  index_to_photoID[i] = photo_IDs[i]

#dictionary: index(value) <-- photo_ID(key) 
photoID_to_index = {}
for i in range(0, len(photo_IDs)):
  photoID_to_index[photo_IDs[i]] = i

### Photos folder안에 있는 photo_ID만 남기기

In [10]:
#Photos folder안에 있는 것만 남기기

print(len(os.listdir(photo_path))) #4978
sub_photo_ID = []

# get photo_ID list in Photos folder
for index,filename in enumerate(os.listdir(photo_path)):
  if filename.endswith(".jpg"): #except folder. only image files
      file = filename.rstrip(".jpg")
      sub_photo_ID.append(int(file))
print(len(sub_photo_ID)) #4851

#Photos folder안에 있는 photo_ID list를 dataframe에서 인덱스 list로
sub_phto_ID_index = []
for idx, i in enumerate(sub_photo_ID):
  index = photoID_to_index[i]
  sub_phto_ID_index.append(index)

print(len(sub_phto_ID_index)) #4851

4978
4851
4851


In [11]:
new_df = rmdup_metadata.loc[[i for idx, i in enumerate(sub_phto_ID_index)], :]
new_df #4851 rows × 6 columns

,photo_ID,user_ID,latitude,longitude,datetime,-1
9175,3286772707,24537256@N02,47.620483,-122.349286,2/17/2009 10:16:21 AM,-1
9177,3286773249,24537256@N02,47.620483,-122.349286,2/17/2009 10:16:51 AM,-1
9191,3287135921,61565201@N00,47.608535,-122.337666,2/17/2009 2:28:05 PM,-1
9193,3287136491,61565201@N00,47.608535,-122.337666,2/17/2009 2:28:22 PM,-1
9194,3287136871,61565201@N00,47.608535,-122.337666,2/17/2009 2:28:33 PM,-1
...,...,...,...,...,...,...
502199,8652137449,66356066@N02,47.607500,-122.343334,4/15/2013 7:10:22 PM,-1
502224,8653224746,66356066@N02,47.620500,-122.349000,4/15/2013 7:05:25 PM,-1
502247,8653225900,66356066@N02,47.620333,-122.349167,4/15/2013 7:05:51 PM,-1
502218,8653237196,66356066@N02,47.607500,-122.343334,4/15/2013 7:10:12 PM,-1


In [12]:
#reset index in dataframe
new_df = new_df.reset_index(drop=True)
new_df

,photo_ID,user_ID,latitude,longitude,datetime,-1
0,3286772707,24537256@N02,47.620483,-122.349286,2/17/2009 10:16:21 AM,-1
1,3286773249,24537256@N02,47.620483,-122.349286,2/17/2009 10:16:51 AM,-1
2,3287135921,61565201@N00,47.608535,-122.337666,2/17/2009 2:28:05 PM,-1
3,3287136491,61565201@N00,47.608535,-122.337666,2/17/2009 2:28:22 PM,-1
4,3287136871,61565201@N00,47.608535,-122.337666,2/17/2009 2:28:33 PM,-1
...,...,...,...,...,...,...
4846,8652137449,66356066@N02,47.607500,-122.343334,4/15/2013 7:10:22 PM,-1
4847,8653224746,66356066@N02,47.620500,-122.349000,4/15/2013 7:05:25 PM,-1
4848,8653225900,66356066@N02,47.620333,-122.349167,4/15/2013 7:05:51 PM,-1
4849,8653237196,66356066@N02,47.607500,-122.343334,4/15/2013 7:10:12 PM,-1


### dictionary (photo_ID <-> index in new_df)

In [13]:
n_photo_IDs = new_df.photo_ID.values
#print(type(photo_IDs))

#dictionary: index(key) --> photo_ID(value) 
n_index_to_photoID = {}
for i in range(0,len(n_photo_IDs)):
  n_index_to_photoID[i] = n_photo_IDs[i]

#dictionary: index(value) <-- photo_ID(key) 
n_photoID_to_index = {}
for i in range(0, len(n_photo_IDs)):
  n_photoID_to_index[n_photo_IDs[i]] = i

## geo matrix 만들기 -> meanshift clustering

[

[latitiude0,longitude0],

[latitiude1,longitude1]

]



In [14]:
latitude = new_df.latitude.values
longitude = new_df.longitude.values

#print(latitude[0])

# create 0 matrix
X = [[0 for x in range(2)] for x in range(len(latitude))]

#print(X.shape)

for i in range(0, len(latitude)):
  X[i][0] = float(latitude[i])
  X[i][1] = float(longitude[i])

X = np.array(X) #순서 중요!
print(X.shape)

(4851, 2)


# Meanshift clustering

In [15]:
from sklearn.cluster import MeanShift

geo_clustering = MeanShift(bandwidth=0.0015,bin_seeding = True)#, cluster_all = False) #,cluster_all = False)  #bin_seeding = True, cluster_all = False) #0.0123->397개 // 0.015->257개
#bandwidth=0.0015 -> 127개
geo_clustering.fit(X)
geo_cluster_label = geo_clustering.labels_

labels_unique = np.unique(geo_cluster_label)
print(labels_unique)
n_cluster = len(labels_unique)

print("number of geo-clusters: ", n_cluster)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126]
number of geo-clusters:  127


In [16]:
geo_cluster_label

array([0, 0, 5, ..., 0, 7, 7], dtype=int64)

## Geo Cluster 결과대로 폴더 만들어 copy한 후 눈으로 결과 확인하기

In [17]:
import shutil

photo_path = os.path.join(os.getcwd(),'Photos')

count_num = 0 #count number of all contents of geo_clusters

# 폴더 만들기 + 해당하는 파일 찾아 카피
for idx, i in enumerate(labels_unique):
    #index_list = []
    folder_name = "geo_cluster"+ str(i) # label number
    print("folder_name is", folder_name)
    folder_path = os.path.join(photo_path, folder_name)                              
    if(os.path.isdir(folder_path) == False):
        os.mkdir(folder_path)                  ## 제출 시 주석 해제  # 폴더 생성 코드             
      #폴더 지우는 코드
      #os.rmdir(folder_path)
    index_cluster_k = []
    for j in range(len(geo_cluster_label)):
        if geo_cluster_label[j] == i: #i = k (unique label)
            index_cluster_k.append(j)
    
    #photoID value
    labelmk_ID = []
    for index, j in enumerate(index_cluster_k):
        labelmk_ID.append(n_index_to_photoID[j]) # 해당하는 포토 아이디 찾기 코드
    print(len(labelmk_ID))
    count_num += len(labelmk_ID)
    
    # 제출 시 주석 해제
    for index, photoID in enumerate(labelmk_ID):
        file_name = str(photoID)+'.jpg'
        src_dir = os.path.join(photo_path, file_name)
        dst_dir = os.path.join(photo_path, folder_name, file_name)
        shutil.copyfile(src_dir, dst_dir)#shutil.move(src_dir, dst_dir) # 해당하는 포토 파일 명 찾아 폴더 안에 이동 
    
#geo_cluster내의 photo_IDs array{i}로 저장하기 -> tag values구할 때 사용
# 주석 해제하기
for i in range(n_cluster):
    globals()['array{}'.format(i)] = []
    folder_name = "geo_cluster"+ str(i) # label number
    #print("folder_name is", folder_name)
    folder_path = os.path.join(photo_path, folder_name)  
    for idx, j in enumerate(os.listdir(folder_path)):
        if j.endswith(".jpg"):
            globals()['array{}'.format(i)].append(j.rstrip(".jpg"))
    #print(len(globals()['array{}'.format(i)]))
#print(array0)

print("count_num", count_num)
    

folder_name is geo_cluster0
344
folder_name is geo_cluster1
278
folder_name is geo_cluster2
263
folder_name is geo_cluster3
265
folder_name is geo_cluster4
221
folder_name is geo_cluster5
186
folder_name is geo_cluster6
183
folder_name is geo_cluster7
187
folder_name is geo_cluster8
179
folder_name is geo_cluster9
150
folder_name is geo_cluster10
149
folder_name is geo_cluster11
148
folder_name is geo_cluster12
122
folder_name is geo_cluster13
97
folder_name is geo_cluster14
94
folder_name is geo_cluster15
92
folder_name is geo_cluster16
88
folder_name is geo_cluster17
97
folder_name is geo_cluster18
80
folder_name is geo_cluster19
73
folder_name is geo_cluster20
71
folder_name is geo_cluster21
71
folder_name is geo_cluster22
61
folder_name is geo_cluster23
59
folder_name is geo_cluster24
57
folder_name is geo_cluster25
53
folder_name is geo_cluster26
53
folder_name is geo_cluster27
43
folder_name is geo_cluster28
48
folder_name is geo_cluster29
45
folder_name is geo_cluster30
42
folde

In [18]:
# import tensorflow as tf
# print(tf.__version__)

# visual clustering

In [19]:
# using VGG16 -> extract feature vectors
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
# from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.decomposition import PCA


model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

## for loop으로 구현... 모든 geo_cluster에 대해

In [20]:
### 0-12 geo clusters(which have more than 100 images)
for i in range(13): 
# for i in range(2,n_cluster): 
    geo_cluster_path = os.path.join(photo_path, 'geo_cluster'+ str(i))#'/content/drive/MyDrive/geo_cluster0'
    print("geo_cluster_path is {}".format(geo_cluster_path))
    # # change the working directory to the path where the images are located
    os.chdir(geo_cluster_path)

    # this list holds all the image filename
    images = []

    # creates a ScandirIterator aliased as files
    with os.scandir(geo_cluster_path) as files:
    # loops through each file in the directory
        for file in files:
            if file.name.endswith('.jpg'):
            # adds only the image files to the flowers list
                images.append(file.name)
    print("len of image is {}".format(len(images))) 
#     print(images)   

    data = {}
    txt_file_name = 'geo'+str(i)+'_featurevectors.txt'
    txt_file_path = os.path.join(photo_path, txt_file_name) 
    print("txt_file_path is {}".format(txt_file_path))
    #r"CHANGE TO A LOCATION TO SAVE FEATURE VECTORS"

    # lop through each image in the dataset
    for image in images:
        # try to extract the features and update the dictionary
        try:
            feat = extract_features(image,model)
            data[image] = feat
        # if something fails, save the extracted features as a pickle file (optional)
        except:
            with open(txt_file_path,'wb') as file:
                pickle.dump(data,file)

    # get a list of the filenames
    filenames = np.array(list(data.keys()))

    # get a list of just the features
    feat = np.array(list(data.values()))

    # reshape so that there are n samples of 4096 vectors
    feat = feat.reshape(-1,4096)
#     print(feat)
    print("len of data is {}".format(len(data)))

    # reduce the amount of dimensions in the feature vector
    pca = PCA(n_components=100, random_state=22) #n_components = 100, random_state=22
    pca.fit(feat)
    x = pca.transform(feat)

    # cluster feature vectors

    visual_clustering = MeanShift(bandwidth=40,bin_seeding = True,cluster_all = True)  #n_components = 100, bandwidth=40 -> 4개
    visual_clustering.fit(x)
    visual_cluster_label = visual_clustering.labels_
    # print(geo_cluster_label)
    labels_unique = np.unique(visual_cluster_label)
    #print(labels_unique)
    vn_cluster = len(labels_unique)

    print("number of {}th visual-clusters: {}".format(i, vn_cluster))

    #remove 주석처리
    # holds the cluster id and the images { id: [images] }
    groups = {}
    for file, cluster in zip(filenames, visual_cluster_label):
        if cluster not in groups.keys():
            groups[cluster] = []
            groups[cluster].append(file)
        else:
            groups[cluster].append(file)

    for i in groups.keys():
      print(len(list(groups[i])))

    count_num = 0 #count number of all contents of geo_clusters

    for j in groups.keys():
      folder_name = "visual_cluster"+str(j)
      print("folder_name is", folder_name)
      folder_path = os.path.join(geo_cluster_path, folder_name)                              
      if(os.path.isdir(folder_path) == False):
          os.mkdir(folder_path)                  ## 제출 시 주석 해제  # 폴더 생성 코드    
      jpglist = []
      jpglist = list(groups[j])
      print("num_of_photos is {}".format(len(jpglist)))
      for jpg in jpglist:
        src_dir = os.path.join(geo_cluster_path, jpg)
        dst_dir = os.path.join(geo_cluster_path, folder_name, jpg)
        shutil.copyfile(src_dir, dst_dir) #shutil.copyfile(src_dir, dst_dir) # 해당하는 포토 파일 명 찾아 폴더 안에 이동 
print("all done")

geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster0
len of image is 344
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo0_featurevectors.txt
len of data is 344
number of 0th visual-clusters: 4
179
74
67
24
folder_name is visual_cluster0
num_of_photos is 179
folder_name is visual_cluster1
num_of_photos is 74
folder_name is visual_cluster3
num_of_photos is 67
folder_name is visual_cluster2
num_of_photos is 24
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster1
len of image is 278
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo1_featurevectors.txt
len of data is 278
number of 1th visual-clusters: 4
139
41
92
6
folder_name is visual_cluster0
num_of_photos is 139
folder_name is visual_cluster2
num_of_photos is 41
folder_name is visual_cluster1
num_of_photos is 92
folder_name is visual_cluster3
num_of_photos is 6
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster2
len of

ValueError: n_components=100 must be between 0 and min(n_samples, n_features)=97 with svd_solver='full'

In [21]:
# print(n_cluster)
for i in range(13,32): #13-31 geo clusters(which have more than 40 images)
    geo_cluster_path = os.path.join(photo_path, 'geo_cluster'+ str(i))#'/content/drive/MyDrive/geo_cluster0'
    print("geo_cluster_path is {}".format(geo_cluster_path))
    # # change the working directory to the path where the images are located
    os.chdir(geo_cluster_path)

    # this list holds all the image filename
    images = []

    # creates a ScandirIterator aliased as files
    with os.scandir(geo_cluster_path) as files:
    # loops through each file in the directory
        for file in files:
            if file.name.endswith('.jpg'):
            # adds only the image files to the flowers list
                images.append(file.name)
    print("len of image is {}".format(len(images))) 
#     print(images)   

    data = {}
    txt_file_name = 'geo'+str(i)+'_featurevectors.txt'
    txt_file_path = os.path.join(photo_path, txt_file_name) ## need to be changed
    print("txt_file_path is {}".format(txt_file_path))
    #p = '/content/drive/MyDrive/geo0_featurevectors.txt'    #r"CHANGE TO A LOCATION TO SAVE FEATURE VECTORS"

    # lop through each image in the dataset
    for image in images:
        # try to extract the features and update the dictionary
        try:
            feat = extract_features(image,model)
            data[image] = feat
        # if something fails, save the extracted features as a pickle file (optional)
        except:
            with open(txt_file_path,'wb') as file:
                pickle.dump(data,file)

    # get a list of the filenames
    filenames = np.array(list(data.keys()))

    # get a list of just the features
    feat = np.array(list(data.values()))

    # reshape so that there are n samples of 4096 vectors
    feat = feat.reshape(-1,4096)
#     print(feat)
    print("len of data is {}".format(len(data)))

    # reduce the amount of dimensions in the feature vector
    pca = PCA(n_components=40, random_state=22) #n_components = 100, random_state=22 ### NEED to change
    pca.fit(feat)
    x = pca.transform(feat)

    # cluster feature vectors

    visual_clustering = MeanShift(bandwidth=40,bin_seeding = True,cluster_all = True)  #n_components = 100, bandwidth=40 -> 4개
    visual_clustering.fit(x)
    visual_cluster_label = visual_clustering.labels_
    # print(geo_cluster_label)
    labels_unique = np.unique(visual_cluster_label)
    #print(labels_unique)
    vn_cluster = len(labels_unique)

    print("number of {}th visual-clusters: {}".format(i, vn_cluster))

    #remove 주석처리
    # holds the cluster id and the images { id: [images] }
    groups = {}
    for file, cluster in zip(filenames, visual_cluster_label):
        if cluster not in groups.keys():
            groups[cluster] = []
            groups[cluster].append(file)
        else:
            groups[cluster].append(file)

    for i in groups.keys():
      print(len(list(groups[i])))

    count_num = 0 #count number of all contents of geo_clusters

    for j in groups.keys():
      folder_name = "visual_cluster"+str(j)
      print("folder_name is", folder_name)
      folder_path = os.path.join(geo_cluster_path, folder_name)                              
      if(os.path.isdir(folder_path) == False):
          os.mkdir(folder_path)                  ## 제출 시 주석 해제  # 폴더 생성 코드    
      jpglist = []
      jpglist = list(groups[j])
      print("num_of_photos is {}".format(len(jpglist)))
      for jpg in jpglist:
        src_dir = os.path.join(geo_cluster_path, jpg)
        dst_dir = os.path.join(geo_cluster_path, folder_name, jpg)
        shutil.copyfile(src_dir, dst_dir) #shutil.copyfile(src_dir, dst_dir) # 해당하는 포토 파일 명 찾아 폴더 안에 이동 
print("all done")

geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster13
len of image is 97
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo13_featurevectors.txt
len of data is 97
number of 13th visual-clusters: 5
27
47
11
4
8
folder_name is visual_cluster1
num_of_photos is 27
folder_name is visual_cluster0
num_of_photos is 47
folder_name is visual_cluster2
num_of_photos is 11
folder_name is visual_cluster4
num_of_photos is 4
folder_name is visual_cluster3
num_of_photos is 8
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster14
len of image is 94
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo14_featurevectors.txt
len of data is 94
number of 14th visual-clusters: 4
14
67
5
8
folder_name is visual_cluster1
num_of_photos is 14
folder_name is visual_cluster0
num_of_photos is 67
folder_name is visual_cluster3
num_of_photos is 5
folder_name is visual_cluster2
num_of_photos is 8
geo_cluster_path is C:\Users\dbw21\Deskt

In [66]:
#32-58 geo clusters(which have less than 40 images, more than 10 images)
for i in range(32,58): 
    geo_cluster_path = os.path.join(photo_path, 'geo_cluster'+ str(i))#'/content/drive/MyDrive/geo_cluster0'
    print("geo_cluster_path is {}".format(geo_cluster_path))
    # # change the working directory to the path where the images are located
    os.chdir(geo_cluster_path)

    # this list holds all the image filename
    images = []

    # creates a ScandirIterator aliased as files
    with os.scandir(geo_cluster_path) as files:
    # loops through each file in the directory
        for file in files:
            if file.name.endswith('.jpg'):
            # adds only the image files to the flowers list
                images.append(file.name)
    print("len of image is {}".format(len(images))) 
#     print(images)   

    data = {}
    txt_file_name = 'geo'+str(i)+'_featurevectors.txt'
    txt_file_path = os.path.join(photo_path, txt_file_name) ## need to be changed
    print("txt_file_path is {}".format(txt_file_path))
    #p = '/content/drive/MyDrive/geo0_featurevectors.txt'    #r"CHANGE TO A LOCATION TO SAVE FEATURE VECTORS"

    # lop through each image in the dataset
    for image in images:
        # try to extract the features and update the dictionary
        try:
            feat = extract_features(image,model)
            data[image] = feat
        # if something fails, save the extracted features as a pickle file (optional)
        except:
            with open(txt_file_path,'wb') as file:
                pickle.dump(data,file)

    # get a list of the filenames
    filenames = np.array(list(data.keys()))

    # get a list of just the features
    feat = np.array(list(data.values()))

    # reshape so that there are n samples of 4096 vectors
    feat = feat.reshape(-1,4096)
#     print(feat)
    print("len of data is {}".format(len(data)))

    # reduce the amount of dimensions in the feature vector
    pca = PCA(n_components=10, random_state=22) #n_components = 100, random_state=22 ### NEED to change
    pca.fit(feat)
    x = pca.transform(feat)

    # cluster feature vectors

    visual_clustering = MeanShift(bandwidth=70,bin_seeding = True,cluster_all = True)  #n_components = 100, bandwidth=40 -> 4개
    visual_clustering.fit(x)
    visual_cluster_label = visual_clustering.labels_
    # print(geo_cluster_label)
    labels_unique = np.unique(visual_cluster_label)
    #print(labels_unique)
    vn_cluster = len(labels_unique)

    print("number of {}th visual-clusters: {}".format(i, vn_cluster))

    #remove 주석처리
    # holds the cluster id and the images { id: [images] }
    groups = {}
    for file, cluster in zip(filenames, visual_cluster_label):
        if cluster not in groups.keys():
            groups[cluster] = []
            groups[cluster].append(file)
        else:
            groups[cluster].append(file)

    for i in groups.keys():
      print(len(list(groups[i])))

    count_num = 0 #count number of all contents of geo_clusters

    for j in groups.keys():
      folder_name = "visual_cluster"+str(j)
      print("folder_name is", folder_name)
      folder_path = os.path.join(geo_cluster_path, folder_name)                              
      if(os.path.isdir(folder_path) == False):
          os.mkdir(folder_path)                  ## 제출 시 주석 해제  # 폴더 생성 코드    
      jpglist = []
      jpglist = list(groups[j])
      print("num_of_photos is {}".format(len(jpglist)))
      for jpg in jpglist:
        src_dir = os.path.join(geo_cluster_path, jpg)
        dst_dir = os.path.join(geo_cluster_path, folder_name, jpg)
        shutil.copyfile(src_dir, dst_dir) #shutil.copyfile(src_dir, dst_dir) # 해당하는 포토 파일 명 찾아 폴더 안에 이동 
print("all done")

geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster32
len of image is 31
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo32_featurevectors.txt
len of data is 31
number of 32th visual-clusters: 2
29
2
folder_name is visual_cluster0
num_of_photos is 29
folder_name is visual_cluster1
num_of_photos is 2
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster33
len of image is 33
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo33_featurevectors.txt
len of data is 33
number of 33th visual-clusters: 1
33
folder_name is visual_cluster0
num_of_photos is 33
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster34
len of image is 32
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo34_featurevectors.txt
len of data is 32
number of 34th visual-clusters: 2
31
1
folder_name is visual_cluster0
num_of_photos is 31
folder_name is visual_cluster1
num_of_photos is 1
geo_cluster

In [80]:
#59-126 geo clusters(which have less than 10images)
for i in range(59,127): 
    geo_cluster_path = os.path.join(photo_path, 'geo_cluster'+ str(i))#'/content/drive/MyDrive/geo_cluster0'
    print("geo_cluster_path is {}".format(geo_cluster_path))
    # # change the working directory to the path where the images are located
    os.chdir(geo_cluster_path)

    # this list holds all the image filename
    images = []

    # creates a ScandirIterator aliased as files
    with os.scandir(geo_cluster_path) as files:
    # loops through each file in the directory
        for file in files:
            if file.name.endswith('.jpg'):
            # adds only the image files to the flowers list
                images.append(file.name)
    print("len of image is {}".format(len(images))) 
#     print(images)   

    data = {}
    txt_file_name = 'geo'+str(i)+'_featurevectors.txt'
    txt_file_path = os.path.join(photo_path, txt_file_name) ## need to be changed
    print("txt_file_path is {}".format(txt_file_path))
    #p = '/content/drive/MyDrive/geo0_featurevectors.txt'    #r"CHANGE TO A LOCATION TO SAVE FEATURE VECTORS"

    # lop through each image in the dataset
    for image in images:
        # try to extract the features and update the dictionary
        try:
            feat = extract_features(image,model)
            data[image] = feat
        # if something fails, save the extracted features as a pickle file (optional)
        except:
            with open(txt_file_path,'wb') as file:
                pickle.dump(data,file)

    # get a list of the filenames
    filenames = np.array(list(data.keys()))

    # get a list of just the features
    feat = np.array(list(data.values()))

    # reshape so that there are n samples of 4096 vectors
    feat = feat.reshape(-1,4096)
#     print(feat)
    print("len of data is {}".format(len(data)))

    # reduce the amount of dimensions in the feature vector
    pca = PCA(n_components=len(data)//3 + 1 , random_state=22) #n_components = 100, random_state=22 ### NEED to change
    pca.fit(feat)
    x = pca.transform(feat)

    # cluster feature vectors

    visual_clustering = MeanShift(bandwidth=50,bin_seeding = True,cluster_all = True)  #n_components = 100, bandwidth=40 -> 4개
    visual_clustering.fit(x)
    visual_cluster_label = visual_clustering.labels_
    # print(geo_cluster_label)
    labels_unique = np.unique(visual_cluster_label)
    #print(labels_unique)
    vn_cluster = len(labels_unique)

    print("number of {}th visual-clusters: {}".format(i, vn_cluster))

    #remove 주석처리
    # holds the cluster id and the images { id: [images] }
    groups = {}
    for file, cluster in zip(filenames, visual_cluster_label):
        if cluster not in groups.keys():
            groups[cluster] = []
            groups[cluster].append(file)
        else:
            groups[cluster].append(file)

    for i in groups.keys():
      print(len(list(groups[i])))

    count_num = 0 #count number of all contents of geo_clusters

    for j in groups.keys():
      folder_name = "visual_cluster"+str(j)
      print("folder_name is", folder_name)
      folder_path = os.path.join(geo_cluster_path, folder_name)                              
      if(os.path.isdir(folder_path) == False):
          os.mkdir(folder_path)                  ## 제출 시 주석 해제  # 폴더 생성 코드    
      jpglist = []
      jpglist = list(groups[j])
      print("num_of_photos is {}".format(len(jpglist)))
      for jpg in jpglist:
        src_dir = os.path.join(geo_cluster_path, jpg)
        dst_dir = os.path.join(geo_cluster_path, folder_name, jpg)
        shutil.copyfile(src_dir, dst_dir) #shutil.copyfile(src_dir, dst_dir) # 해당하는 포토 파일 명 찾아 폴더 안에 이동 
print("all done")

geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster59
len of image is 10
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo59_featurevectors.txt
len of data is 10
number of 59th visual-clusters: 1
10
folder_name is visual_cluster0
num_of_photos is 10
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster60
len of image is 10
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo60_featurevectors.txt
len of data is 10
number of 60th visual-clusters: 1
10
folder_name is visual_cluster0
num_of_photos is 10
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster61
len of image is 9
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo61_featurevectors.txt
len of data is 9
number of 61th visual-clusters: 5
2
1
1
4
1
folder_name is visual_cluster1
num_of_photos is 2
folder_name is visual_cluster2
num_of_photos is 1
folder_name is visual_cluster3
num_of_photos is 1
folder_name

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_mean_shift.py:236: UserWarning: Binning data failed with provided bin_size=50.000000, using data points as seeds.
  warnings.warn("Binning data failed with provided bin_size=%f,"


len of data is 7
number of 76th visual-clusters: 3
3
2
2
folder_name is visual_cluster0
num_of_photos is 3
folder_name is visual_cluster1
num_of_photos is 2
folder_name is visual_cluster2
num_of_photos is 2
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster77
len of image is 7
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo77_featurevectors.txt
len of data is 7
number of 77th visual-clusters: 3
5
1
1
folder_name is visual_cluster0
num_of_photos is 5
folder_name is visual_cluster1
num_of_photos is 1
folder_name is visual_cluster2
num_of_photos is 1
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster78
len of image is 7
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo78_featurevectors.txt
len of data is 7
number of 78th visual-clusters: 4
2
3
1
1
folder_name is visual_cluster1
num_of_photos is 2
folder_name is visual_cluster0
num_of_photos is 3
folder_name is visual_cluster3
num_of_photos is 1
f

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_mean_shift.py:236: UserWarning: Binning data failed with provided bin_size=50.000000, using data points as seeds.
  warnings.warn("Binning data failed with provided bin_size=%f,"


len of data is 8
number of 86th visual-clusters: 3
2
5
1
folder_name is visual_cluster1
num_of_photos is 2
folder_name is visual_cluster0
num_of_photos is 5
folder_name is visual_cluster2
num_of_photos is 1
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster87
len of image is 6
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo87_featurevectors.txt
len of data is 6
number of 87th visual-clusters: 3
4
1
1
folder_name is visual_cluster0
num_of_photos is 4
folder_name is visual_cluster1
num_of_photos is 1
folder_name is visual_cluster2
num_of_photos is 1
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster88
len of image is 5
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo88_featurevectors.txt


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:468: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_mean_shift.py:236: UserWarning: Binning data failed with provided bin_size=50.000000, using data points as seeds.
  warnings.warn("Binning data failed with provided bin_size=%f,"


len of data is 2
number of 125th visual-clusters: 2
1
1
folder_name is visual_cluster0
num_of_photos is 1
folder_name is visual_cluster1
num_of_photos is 1
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster126
len of image is 1
txt_file_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo126_featurevectors.txt


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_mean_shift.py:236: UserWarning: Binning data failed with provided bin_size=50.000000, using data points as seeds.
  warnings.warn("Binning data failed with provided bin_size=%f,"


len of data is 1
number of 126th visual-clusters: 1
1
folder_name is visual_cluster0
num_of_photos is 1
all done


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_pca.py:468: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_mean_shift.py:236: UserWarning: Binning data failed with provided bin_size=50.000000, using data points as seeds.
  warnings.warn("Binning data failed with provided bin_size=%f,"


## remove folders = wrong result of visual clustering

In [81]:
# # # # print(array126)
# # # # print(n_cluster)
# import os, glob

for i in range(59,127): #remove wrong result folder of visual clustering
    geo_cluster_path = os.path.join(photo_path, 'geo_cluster'+ str(i))#'/content/drive/MyDrive/geo_cluster0'
    print("geo_cluster_path is {}".format(geo_cluster_path))
    # # change the working directory to the path where the images are located
    os.chdir(geo_cluster_path)
    subfolderlist = list(filter(os.path.isdir, glob.glob('*')))
    print(subfolderlist)
    for j in subfolderlist:
        removepath = os.path.join(geo_cluster_path, j)
        print(removepath)
        shutil.rmtree(removepath)
    
    

# Tag Matching


## read csv file

In [87]:
os.chdir(firstposition)
csv_path = os.path.join(os.getcwd(),'Tag (1).csv')
tag_data = pd.read_csv(csv_path, names = ['photo_ID','num','tag'])

In [88]:
def preprocessing(tag): #영어, 숫자만 남기기
    tag = re.sub('[^a-zA-Z0-9]',"",tag)
    return tag

In [89]:
#tag_data['tag'].map(lambda x: print(type(x)))
print(tag_data.loc[103])

photo_ID               6248446723
num                             5
tag         uploaded:by=instagram
Name: 103, dtype: object


In [90]:
# 모두 소문자로 변경
tag_data['tag'] = tag_data['tag'].str.lower()

# 영어, 숫자만 남기기
tag_data['tag'] = tag_data['tag'].map(lambda x: (preprocessing(str(x))))
                                      

In [91]:
print(tag_data.loc[103])

photo_ID             6248446723
num                           5
tag         uploadedbyinstagram
Name: 103, dtype: object


In [93]:
tag_data #115755 rows × 3 columns

,photo_ID,num,tag
0,8230337045,0,seattle
1,8230337045,1,museum
2,8230337045,2,machine
3,8230337045,3,planes
4,8231397586,0,seattle
...,...,...,...
115750,3521684288,10,450d
115751,3521684288,11,bellevueskyline
115752,8272100050,0,seattle
115753,8272100050,1,guitars


In [94]:
tag_photo_IDs = tag_data.photo_ID.values
tags = tag_data.tag.values
print(tags)

['seattle' 'museum' 'machine' ... 'seattle' 'guitars' 'emp']


In [95]:
#tags에서 가장 빈도수 높은 단어 출력
print(type(tags))
#print(tags)
unique,pos = np.unique(tags,return_inverse=True) #Finds all unique elements and their positions
counts = np.bincount(pos)                     #Count the number of each unique element
maxpos = counts.argmax()                      #Finds the positions of the maximum count
#print(unique)
print(len(unique))
print(unique[maxpos],counts[maxpos])

<class 'numpy.ndarray'>
11102
seattle 7564


## tag에서 가장 빈도수 높은 것 출력
- park, westseattle, bellevue, lake (이런 거 말고 다른 몇개 단어 stopwords로 정의하기)

In [96]:
tagwords = tag_data['tag'].value_counts()
# print(type(tagwords))
# print(tagwords.head(50))

#convert value_counts() output into a data frame
df_tagwords = tag_data['tag'].value_counts().reset_index()
df_tagwords.columns = ['word', 'word_count']
print(df_tagwords)

              word  word_count
0          seattle        7564
1       washington        2958
2              usa         785
3               wa         718
4       convention         667
...            ...         ...
11097        m33gs           1
11098  cloudycloud           1
11099     raincity           1
11100          dii           1
11101    sigma50mm           1

[11102 rows x 2 columns]


## define stopwords

In [97]:
#define stopwords
stopwords = []
not_stopwords = ['park', 'art','lakeunion','bellevue'] #'seattle', 'washington'
for i in range(50):
    word = df_tagwords['word'].values[i]
    if word not in not_stopwords:
        stopwords.append(word)
print(len(stopwords))
print(stopwords)
# result = []
# for word in word_tokens: 
#     if word not in stop_words: 
#         result.append(word)

46
['seattle', 'washington', 'usa', 'wa', 'convention', '2009', 'square', 'redmond', '2012', 'iphoneography', 'uploadedbyinstagram', 'squareformat', 'instagramapp', 'rf', 'unitedstates', '2010', 'con', 'furry', 'rf2012', 'rainfurrest', 'eyefi', 'kinner', 'concerts', 'ratcityrollergirls', '2004', 'rcrg', 'rollerderby', 'westseattle', 'swarali', 'iphone', 'downtown', 'august', 'concert', '', '180550mm', 'june', 'nikon', 'cameraphone', 'canon', 'music', 'live', 'show', 'wedding', 'travel', 'dylandiana', 'water']


In [98]:
condition = (tag_data['photo_ID']  == 8230337045)
print(tag_data[condition])
print(len(tag_data[condition]))

tags = tag_data[condition].tag.values
print(tags)

     photo_ID  num      tag
0  8230337045    0  seattle
1  8230337045    1   museum
2  8230337045    2  machine
3  8230337045    3   planes
4
['seattle' 'museum' 'machine' 'planes']


## tag에서 아까 정의한 46개의 stopwords 제거하기

In [99]:
def removingSW(tag): 
    if tag in stopwords:
        tag = ""
    return tag
tag_data['tag'] = tag_data['tag'].map(lambda x: (removingSW(str(x))))

## dictionary(photo_ID -> array of tag values)

In [100]:
#dictionary: key(photoID) --> value(array of tag values) 
photoID_to_tags = {}
# for i in range(0,len(photo_IDs)):
#   index_to_photoID[i] = photo_IDs[i]

for idx, i in enumerate(tag_photo_IDs):
  condition = (tag_data['photo_ID']  == i)
  photoID_to_tags[i] = tag_data[condition].tag.values
        
# #   tags_wo_stopwords = []
#   for i in range(len(tag_data[condition].tag.values)): #remove stopwords
#     word = tag_data[condition].tag.values[i]
#     if word not in stopwords:
#         tags_wo_stopwords.append(word)
#   photoID_to_tags[i] = tags_wo_stopwords

print(photoID_to_tags)

{8230337045: array(['', 'museum', 'machine', 'planes'], dtype=object), 8231397586: array(['', 'museum', 'thingsinplaces'], dtype=object), 8231397198: array(['museum', 'ruby'], dtype=object), 8230334599: array(['museum'], dtype=object), 8231395940: array(['', 'museum', 'thingsinplaces'], dtype=object), 8231394954: array(['', 'museum', 'plane'], dtype=object), 8231396500: array(['', 'museum', 'ruby', 'thefam'], dtype=object), 8231395532: array(['museum', 'toilet', 'planes', 'thingsinplaces'], dtype=object), 8230335467: array(['', 'museum', 'thingsinplaces'], dtype=object), 8230335655: array(['', 'museum', 'ruby', 'thefam'], dtype=object), 8231394210: array(['museum', 'machine'], dtype=object), 8231394446: array(['museum', 'thingsinplaces'], dtype=object), 8230305023: array(['', 'blackandwhite', 'bw', 'white', 'black', 'horizontal', 'wall',
       'contrast', 'poster', '', '', 'image', '', 'rageagainstthemachine',
       'stockphoto', 'rm', 'ratm', 'royaltyfree', 'rightsmanaged'],
      d

In [101]:
print(photoID_to_tags[8230337045]) #remove 'seattle' ->""

['' 'museum' 'machine' 'planes']


# 각 geo_cluster에 있는 사진들의 tag values 구하기

In [34]:
print(array25)

['6190676583', '6190679933', '6190680303', '6190684319', '6190685763', '6190686027', '6190686475', '6190688279', '6190688749', '6190689219', '6190690599', '6190691055', '6190691465', '6190695301', '6190696751', '6190697439', '6190698033', '6190698647', '6190701849', '6190703911', '6190705835', '6190706903', '6190707411', '6190707851', '6190708779', '6190709241', '6190710333', '6190710891', '6190713585', '6190713959', '6190716111', '6191193806', '6191194300', '6191196844', '6191197334', '6191201534', '6191203980', '6191204410', '6191205930', '6191206348', '6191208730', '6191209232', '6191212184', '6191215052', '6191221066', '6191221494', '6191222772', '6191224670', '6191225036', '6191229954', '6191232986', '6191234988', '6191235400']


In [35]:
tag_array25 = []
for idx, ID in enumerate(array25): #ID: string value
    if(int(ID) in photoID_to_tags): #check ID is key of photoID_to_tags
        tag_array25.extend(photoID_to_tags[int(ID)])
        print("ID: {} tags: {}".format(ID, photoID_to_tags[int(ID)]))
print("tag_array25: {}".format(tag_array25))
    

ID: 6190676583 tags: ['seattle' 'silly' 'art' 'funny' 'paintings' 'greenwood' 'drawings'
 'figurines' 'talent' 'mad' 'sketches' 'talented' 'madmagazine'
 'urbanlightstudios']
ID: 6190679933 tags: ['seattle' 'silly' 'art' 'funny' 'paintings' 'greenwood' 'drawings'
 'figurines' 'talent' 'mad' 'sketches' 'talented' 'madmagazine'
 'urbanlightstudios']
ID: 6190680303 tags: ['seattle' 'silly' 'art' 'funny' 'paintings' 'greenwood' 'drawings'
 'figurines' 'talent' 'mad' 'sketches' 'talented' 'madmagazine'
 'urbanlightstudios']
ID: 6190684319 tags: ['seattle' 'silly' 'art' 'funny' 'paintings' 'greenwood' 'drawings'
 'figurines' 'talent' 'mad' 'sketches' 'talented' 'madmagazine'
 'urbanlightstudios']
ID: 6190685763 tags: ['seattle' 'silly' 'art' 'funny' 'paintings' 'greenwood' 'drawings'
 'figurines' 'talent' 'mad' 'sketches' 'talented' 'madmagazine'
 'urbanlightstudios']
ID: 6190686027 tags: ['seattle' 'silly' 'art' 'funny' 'paintings' 'greenwood' 'drawings'
 'figurines' 'talent' 'mad' 'sketche

In [37]:
tag_array25 = []
for idx, ID in enumerate(array25): #ID: string value
    if(int(ID) in photoID_to_tags): #check ID is key of photoID_to_tags
        tag_array25.extend(photoID_to_tags[int(ID)])
        #print("ID: {} tags: {}".format(ID, photoID_to_tags[int(ID)]))
print("tag_array25: {}".format(list(set(tag_array25))))
    

tag_array25: ['', 'figurines', 'talented', 'funny', 'paintings', 'urbanlightstudios', 'greenwood', 'seattle', 'mad', 'ryanhenryward', 'pagan', 'art', 'starheadboy', 'davebloomfield', 'henry', 'drawings', 'madmagazine', 'sketches', 'madspongebob', 'spongebob', 'silly', 'talent']


### 각 cluster에서 중복 없앤 tag array 보기(print)

In [40]:
for i in range(n_cluster):
    globals()['tag_array{}'.format(i)] = []
    for idx, ID in enumerate(globals()['array{}'.format(i)]): #ID: string value
        if(int(ID) in photoID_to_tags): #check ID is key of photoID_to_tags
            globals()['tag_array{}'.format(i)].extend(photoID_to_tags[int(ID)])
            #print("ID: {} tags: {}".format(ID, photoID_to_tags[int(ID)]))
    print("tag_array{}: {}".format(i,list(set(globals()['tag_array{}'.format(i)]))))

tag_array0: ['', 'tubes', 'center', 'boat', 'cessna172pn62879', 'smithsonianchannel', 'lauraling', 'spaceneedlelookout', 'speed', 'geolat4762088889', 'interior', 'building', 'worldsfair', 'hipstamatic', 'union', 'concrete', 'frankgehry', 'ripmichaeljackson', 'aerial', 'kristinrowefinkbeiner', 'outdoorart', 'd50', 'johntwilliams', 'twinlensreflex', 'westcoast', 'rebcsea', 'northamerica', 'arches', 'kexp', 'light', 'harbortour', 'aerialphotography', 'monorail', 'fall', 'hadden', 'spaceneedleseattle', 'sp', 'geotagged', 'portrait', 'niall', 'boats', 'coliseum', 'seattleskyline', 'control', 'kissing', 'kodakportra160vc', 'garden', 'landmark', 'boattrip', 'deck', 'rebarcamp', 'seattlecenter', 'champagnechampagne', 'tower', 'aerialamerica', 'transport', 'greenspace', 'gavinnewsom', 'journey', 'minoltaautocordmxs', 'nighttime', 'rise', 'pacificcoast', 'hefe', 'sam', 'darnellstrom', 'zoom', 'geolat4762041667', 'coast', 'century21', 'nikonians', 'observation', 'futuristic', 'cymbalseatguitars',

## Seattle landmark dictionary 만들기

In [156]:
# space needle, safecofield, pikeplacemarket, seattleaquarium, 
# centurylinkfield(lumen field), sam(seattleartsmuseum), volunteerpark, ballardlocks, 
# virginiav, newcastlebeachpark, spacemuseum...
# urbanlightstudios

dict_landmarks = {'space needle': ['spaceneedle', 'space', 'needle'], 'safeco field': ['safecofield', 'safeco'], 'pike place market': ['pikeplace', 'pikeplacemarket','pike','market'],
'centurylink field':['centurylinkfield','centurylink','centurylinkstadium', 'stadium', 'stadiumns','instagramvenuenamecenturylinkfield'], 'sam': ['sam','seattleartmuseum'], 'volunteerpark': ['volunteerpark','volunteerparkconservatoryplants'],'ballard locks':['ballardlocks'], 'virginia V':['virginiav'], 'newcastle beach park': ['newcastle'], 'Aviation space museum':['space museum','boeingmuseumofflight','museumofflight'],
                 'Monorail':['monorail'], 'seattleaquarium': ['seattleaquarium']}

# print(dict_landmarks)

# print(dict_landmarks['space needle'])

keys = list(dict_landmarks.keys())
print(keys)
# for idx, values in enumerate(dict_landmarks.values()):
# #     print(values)
# #     print(idx, values)
#     print("key is ", keys[idx])
#     for j in range(len(values)):
#         print(values[j])
# Photos folder안에 landmarks folder만들기
os.chdir(firstposition)
print(os.getcwd())
landmark_path = os.path.join(os.getcwd(),'Photos','landmarks')
if(os.path.isdir(landmark_path) == False):
          os.mkdir(landmark_path)
        
for key in keys:
    eachlandmark_path = os.path.join(landmark_path, key)
    if(os.path.isdir(eachlandmark_path) == False):
          os.mkdir(eachlandmark_path)
    

['space needle', 'safeco field', 'pike place market', 'centurylink field', 'sam', 'volunteerpark', 'ballard locks', 'virginia V', 'newcastle beach park', 'Aviation space museum', 'Monorail', 'seattleaquarium']
C:\Users\dbw21\Desktop\jupyter-notebook


In [130]:
# for key in keys:
#     eachlandmark_path = os.path.join(landmark_path, key)
#     if(os.path.isdir(eachlandmark_path) == False):
#           os.mkdir(eachlandmark_path)

C:\Users\dbw21\Desktop\jupyter-notebook


# landmark 찾고 폴더에 모으고 폴더 이름 그에 맞게... + 각 폴더 위치를 photos안에 landmarks 폴더 안으로...?

In [107]:
# 각 geo_cluster folder안의 visual_cluster folders의 tag구해서 landmark dictionary의 values에 있는 지(완전 같지 않아도 유사한지) 매칭,,,?
    # 만약 해당한다면 그 tag를 갖는 사진이 속하는 visual_cluster를(아니면 사진만,,,?) 
        # 그 dictionary key값을 folder name으로 하는 folder가 photos에 없을 경우 만들기
        # 그 landmarks folder로 옮기기
        
#visual cluster의 tag set구하기        
for i in range(0,127): #Geoclusters
    geo_cluster_path = os.path.join(photo_path, 'geo_cluster'+ str(i))
    print("geo_cluster_path is {}".format(geo_cluster_path))
    # # change the working directory to the path where the images are located
    os.chdir(geo_cluster_path)
    subfolderlist = list(filter(os.path.isdir, glob.glob('*')))
    #print(subfolderlist)
    for index,visualcluster in enumerate(subfolderlist): #Visual Clusters
        visualclusters = os.path.join(geo_cluster_path, visualcluster)
        globals()['tag_geo{}_vc{}'.format(i,index)] = []
        #print(visualclusters)
        for idx, j in enumerate(os.listdir(visualclusters)):
            if j.endswith(".jpg"):
                file = j.rstrip(".jpg")
                if(int(file) in photoID_to_tags): #check ID is key of photoID_to_tags
                    globals()['tag_geo{}_vc{}'.format(i,index)].extend(photoID_to_tags[int(file)])
        print("tag_geo{}_vc{} is {}".format(i,index, list(set(globals()['tag_geo{}_vc{}'.format(i,index)])) ))


C:\Users\dbw21\Desktop\jupyter-notebook
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster0
tag_geo0_vc0 is ['', 'andrew', 'shadow', 'hipstamatic', 'photocontest', 'bornanchors', 'rebc', 'pacificsciencecenter', 'unitedstatesofamerica', 'niall', 'high', 'city', '1970s', 'reflection', 'project', 'views', 'observation', 'union', 'd50', 'wifi', 'rebcsea', 'technique', 'work', 'geotagged', 'kodakportra160vc', 'cessna172pn62879', 'citizenuniversity', 'japandroids', 'emp', 'geolon12234907778', 'bestcamera', 'sculpture', 'honorpole', 'mtrainier', 'wings', 'd300', 'jungle', 'spaceneedlelookout', 'festivals', 'geolat4762041667', 'totempole', 'nikonians', 'network', 'aerialamerica', 'dark', 'architecture', '365', 'kexpbbq', '366', 'joseantoniovargas', 'skyline', 'life', 'davidschultzphotographycom', 'century21exposition', 'concrete', 'night', 'frankgehry', 'space', 'muralamphitheatre', 'leaf', 'darnellstrom', 'wendyspencer', 'seahawks', 'rails', 'arches', 'd7000', 'dec

tag_geo3_vc1 is ['', 'chriseggers', 'zacharycrooks', 'brandygrouthumphrey', 'timgrout', 'lauravandeputte', 'veraswettcrooks', 'brianhumphrey', 'jeffeggers', 'nicoleeggerstoskey', 'brandyhumphries', 'brianagrout']
tag_geo3_vc2 is ['', 'zacharycrooks', 'lauravandeputte']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster4
tag_geo4_vc0 is ['museum', '', 'nwsm', 'hdr', 'uspsjet', 'fisheye', 'kingcounty', 'httpwwwmuseumofflightorgpressnorthwestscalemodelersmassiveshowforce', 'tankmk1', 'plane', 'model', 'r2d2', 'carrier', 'robotarm', 'fire', 'jetliners', 'upsaviation', 'spaceshuttle', 'jetliner', 'sr71', '042x', 'stealthbomber', 'northwestscalemodelersmassiveshowoftheforce', 'rutanvoyager', 'spaceshuttlecolumbia', 'nimitz', 'flight', 'boeingfield', 'bobjacobsen', 'ups', 'aviation', 'ussenterprise', 'museumofflight', 'washingtonstate', 'airport', 'mikemorrow', 'boeingmuseumofflight', 'httpswwwmuseumofflightorg', 'smcpda1855mmf3556al', 'viper', 'island', 'supergupp

tag_geo9_vc0 is ['', 'woodenboat', 'allurecruise', '1928', 'marcyportraits', 'lakeunion', 'lake', 'cruise']
tag_geo9_vc1 is ['', 'woodenboat', 'allurecruise', '1928', 'marcyportraits', 'lakeunion', 'lake', 'cruise']
tag_geo9_vc2 is ['', 'woodenboat', 'allurecruise', '1928', 'lakeunion', 'lake', 'cruise']
tag_geo9_vc3 is ['', 'woodenboat', 'allurecruise', '1928', 'lakeunion', 'lake', 'cruise']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster10
tag_geo10_vc0 is ['', 'snow', 'valerie', 'town', 'cycling', 'camera', 'dassera', 'raksha', 'hair', 'dandiya', 'apple', 'sujal', 'messy', 'parade', 'halloween', 'pyjama', 'march', 'ruturaj', 'dada', 'anjali', 'peekaboo', 'priya', 'extended', 'raas', 'ishani', 'sage', 'july', 'handed', 'summer', 'first', 'work', 'poornima', 'friends', 'rita', 'mami', '2005', 'loved', 'ashish', '31supstsup', 'bhadresh', 'neel', 'jayant', 'lovely', 'evening', 'dadi', 'shon', 'kurta', 'gauri', 'ashok', 'dress', 'yard', 'bubbles', 'probably

tag_geo19_vc1 is ['', 'pugetsound', 'condo', 'beachdrive', 'bulkhead', 'barge', 'tug', 'alki']
tag_geo19_vc2 is ['', 'pugetsound', 'condo', 'beachdrive', 'bulkhead', 'barge', 'tug', 'alki']
tag_geo19_vc3 is ['', 'pugetsound', 'condo', 'beachdrive', 'bulkhead', 'barge', 'tug', 'alki']
tag_geo19_vc4 is ['', 'pugetsound', 'condo', 'beachdrive', 'bulkhead', 'barge', 'tug', 'alki']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster20
tag_geo20_vc0 is ['', 'afsvrzoomnikkor70200mmf28gifed', 'afszoomnikkor2470mmf28ged']
tag_geo20_vc1 is ['', 'afsvrzoomnikkor70200mmf28gifed', 'afszoomnikkor2470mmf28ged']
tag_geo20_vc2 is ['', 'afsvrzoomnikkor70200mmf28gifed', 'afszoomnikkor2470mmf28ged']
tag_geo20_vc3 is ['', 'afsvrzoomnikkor70200mmf28gifed', 'afszoomnikkor2470mmf28ged']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster21
tag_geo21_vc0 is ['', 'seacomp09', 'decompression', 'burningmanculture', 'events', 'parties', 'seacompression', 'seaco

tag_geo32_vc0 is ['', 'black', 'red', 'universityfowashingtoninformationschool', 'shadow', 'suite015', 'forpedestriansaga', 'cute', 'girl', 'beautiful', 'outdoor', 'gadis', 'c', 'offwhite', 'sexy', 'female', 'xinh', 'campus', 'model', 'tilt', 'beauty', 'smile', 'lifestyle', 'portrait', 'seattleflickrites', 'hermosa', 'uwischool', 'maganda', 'uwgarage11', 'meanyhallfortheperformingarts', 'shadows', 'universityofwashington', 'ischool', 'uwinformationschool', 'style', 'belle', 'vivianluu', 'kirei', 'sign', 'mse100320', 'informationschool', 'woman', 'cewek', 'p', 'linda', 'meanyhall', 'cantik', 'pink', 'universityofwashingtonischool', 'strobist', 'spoonerism', 'pose', 'lady', 'garageshoot11', 'em', 'lamp', 'fashion', 'graffiti', 'dress', 'garage', 'aisharose', 'vietnamese', 'signs', 'pretty', 'manis', 'heels', 'angle', 'vignette', 'gi', 'uwgarage100320', 'suzzallolibrary', 'cathedral', 'library', 'uw', 'white', 'asian']
tag_geo32_vc1 is ['red', 'seattleflickrites', 'uwgarage11', 'strobist'

tag_geo52_vc0 is ['', 'black', 'chevrolet', 'blue', '1955', '100', 'patina', 'ianpageechols', 'v8media', 'custom', 'austinhealey', 'car', 'diamondt', 'yellow', 'whitezombie', 'pickup', 'chevy', 'electric', 'datsun', 'greenwoodcarshow', 'plymouth', 'dragracing', 'rust', 'white']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster53
tag_geo53_vc0 is ['', 'park', 'discover']
tag_geo53_vc1 is ['', 'park', 'discover']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster54
tag_geo54_vc0 is ['zachary', 'crooks', '58174mm']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster55
tag_geo55_vc0 is ['', 'bowling', 'imperiallanes', 'score', 'dad', 'mothersday', 'lanes', 'raw', 'bowlingalley', 'bowlingballs', 'mom', 'charlie']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster56
tag_geo56_vc0 is ['', 'ukelele', 'tinymanhood', 'jocelynehoughton', 'me', 'bluemoontavern', 'tinymanhoodbluemoontaver

tag_geo79_vc2 is ['bing', 'greengeeks', 'chrispirillo']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster80
tag_geo80_vc0 is ['', 'festival', 'lakeunion', 'woodenboats']
tag_geo80_vc1 is ['', 'festival', 'lakeunion', 'woodenboats']
tag_geo80_vc2 is ['', 'festival', 'lakeunion', 'woodenboats']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster81
tag_geo81_vc0 is ['', 'capitolhillblockparty', 'kexp', 'ghostlandobservatory']
tag_geo81_vc1 is ['', 'capitolhillblockparty', 'craftspells', 'kexp']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster82
tag_geo82_vc0 is ['goats', 'dairy', 'spring', 'dropstonefarms', 'minimanchas', 'dairygoats']
tag_geo82_vc1 is ['goats', 'dairy', 'spring', 'dropstonefarms', 'minimanchas', 'dairygoats']
tag_geo82_vc2 is ['goats', 'dairy', 'spring', 'dropstonefarms', 'minimanchas', 'dairygoats']
tag_geo82_vc3 is ['goats', 'dairy', 'spring', 'dropstonefarms', 'minimanchas', 'dairygo

tag_geo117_vc0 is ['', 'spaceneedle', 'clouds']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster118
tag_geo118_vc0 is ['', 'place', 'd700', 'dopplrexplore2rd1', 'starbucks', 'kenyan', 'clean', 'night', '18g', 'jobless', 'food', 'market', 'evening', 'hope', 'money', '50mm', 'pikeplacemarket', 'pike', 'homeless']
tag_geo118_vc1 is ['', 'spaceneedle', 'needle', 'space']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster119
tag_geo119_vc0 is ['', 'pugetsound', 'spaceneedle', 'skyline', 'bluesky', 'seattlebremertonferry', 'waterfront']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster120
tag_geo120_vc0 is ['spaceneedle']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster121
tag_geo121_vc0 is ['', 'ferry', 'needle', 'space']
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster122
tag_geo122_vc0 is ['', 'polarizer', 'filter', 'harborisland', 'stadium', '

In [158]:
# 각 geo_cluster folder안의 visual_cluster folders의 tag구해서 landmark dictionary의 values에 있는 지(완전 같지 않아도 유사한지) 매칭,,,?
    # 만약 해당한다면 그 tag를 갖는 사진이 속하는 visual_cluster를(아니면 사진만,,,?) 
        # 그 dictionary key값을 folder name으로 하는 folder가 photos에 없을 경우 만들기
        # 그 landmarks folder로 옮기기

from distutils.dir_util import copy_tree

#한 visual cluster에서 landmark tag 있으면 전부를 옮기니까 outlier도 같이 옴. 
#그러므로 한 visual cluster에서 landmark tag있으면 그 visual cluster내에 있는 
#각 이미지들(for loop)에 대해 tag구하고 landmark tag있는 이미지만 옮기기 

for geo_index in range(0,127): #Geoclusters #0-127
    geo_cluster_path = os.path.join(photo_path, 'geo_cluster'+ str(geo_index))
    print("geo_cluster_path is {}".format(geo_cluster_path))
    # # change the working directory to the path where the images are located
    os.chdir(geo_cluster_path)
    subfolderlist = list(filter(os.path.isdir, glob.glob('*')))
    #print(subfolderlist)
#     os.chdir(photo_path)
    for visual_index,visualcluster in enumerate(subfolderlist): #Visual Clusters
        for idx, values in enumerate(dict_landmarks.values()):
            for k in range(len(values)):
                if values[k] in globals()['tag_geo{}_vc{}'.format(geo_index,visual_index)]:
                    for img in os.listdir(os.path.join(geo_cluster_path, visualcluster)):
                        ID = img.rstrip(".jpg")
                        if(int(ID) in photoID_to_tags): #check ID is key of photoID_to_tags
                            if values[k] in photoID_to_tags[int(ID)]:
                                src_dir = os.path.join(geo_cluster_path, visualcluster,img)
                                dst_dir = os.path.join(photo_path,'landmarks' ,keys[idx], img)
                                shutil.copyfile(src_dir, dst_dir)  #이미 존재하는 폴더에 복사
print("All done")                    


geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster0
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster1
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster2
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster3
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster4
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster5
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster6
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster7
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster8
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster9
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster10
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster11
geo_cluster_path is C:\Users\dbw21\Des

geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster103
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster104
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster105
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster106
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster107
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster108
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster109
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster110
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster111
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster112
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster113
geo_cluster_path is C:\Users\dbw21\Desktop\jupyter-notebook\Photos\geo_cluster114
geo_cluster_path

In [132]:
print("Hello")

Hello


In [ ]:
# for geo_index in range(0,1): #Geoclusters #0-127
#     geo_cluster_path = os.path.join(photo_path, 'geo_cluster'+ str(geo_index))
#     print("geo_cluster_path is {}".format(geo_cluster_path))
#     # # change the working directory to the path where the images are located
#     os.chdir(geo_cluster_path)
#     subfolderlist = list(filter(os.path.isdir, glob.glob('*')))
#     #print(subfolderlist)
# #     os.chdir(photo_path)
#     for visual_index,visualcluster in enumerate(subfolderlist): #Visual Clusters
#         for idx, values in enumerate(dict_landmarks.values()):
#             for k in range(len(values)):
#                 if values[k] in globals()['tag_geo{}_vc{}'.format(geo_index,visual_index)]:
#                     for img in os.listdir(os.path.join(geo_cluster_path, visualcluster)):
#                         ID = img.rstrip(".jpg")
#                         if(int(ID) in photoID_to_tags): #check ID is key of photoID_to_tags
#                             if values[k] in photoID_to_tags[int(ID)]:
#                                 src_dir = os.path.join(geo_cluster_path, visualcluster,img)
#                                 dst_dir = os.path.join(photo_path,'landmarks' ,keys[idx], img)
#                                 shutil.copyfile(src_dir, dst_dir)  #이미 존재하는 폴더에 복사


# #     for visual_index,visualcluster in enumerate(subfolderlist): #Visual Clusters
# #         for idx, values in enumerate(dict_landmarks.values()):
# #             for k in range(len(values)):
# #                 if values[k] in globals()['tag_geo{}_vc{}'.format(geo_index,visual_index)]:
# #                     for img in os.listdir(os.path.join(geo_cluster_path, visualcluster)):
# #                         src_dir = os.path.join(geo_cluster_path, visualcluster,img)
# #                         dst_dir = os.path.join(photo_path,'landmarks' ,keys[idx], img)
# #                         shutil.copyfile(src_dir, dst_dir)  #이미 존재하는 폴더에 복사
# print("All done")         